In [ ]:
import os
import sys
dirname = os.path.dirname
BASE_DIR = os.path.abspath(dirname(dirname(os.path.abspath('.'))))
print(BASE_DIR)
sys.path.insert(0, BASE_DIR)

In [ ]:
import json

In [ ]:
import warnings; warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

import logging

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [ ]:
from utils.ner_processor import NerProcessor
from utils.input_example_to_tensors import InputExampleToTensors
from utils.utils import prune_examples
from transformers import BertTokenizer

# START

### Tokenizer

In [ ]:
pretrained_model_name = f'{BASE_DIR}/pretrained_models/bert-base-swedish-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name, do_lower_case=False)

### NerProcessor

In [ ]:
dataset_path = f'{BASE_DIR}/datasets/ner/swedish_ner_corpus/'
processor = NerProcessor(dataset_path, tokenizer, do_lower_case=False)

In [ ]:
label_list = processor.get_label_list()
label_list

In [ ]:
train_input_examples_all = processor.get_input_examples('train')

In [ ]:
train_input_examples = prune_examples(train_input_examples_all, ratio=0.01)

### InputExampleToTensors

In [ ]:
samples_transformer = InputExampleToTensors(tokenizer, 
                                            max_seq_length=16, 
                                            label_tuple=tuple(label_list))

### Example

In [ ]:
ex = train_input_examples[2]

In [ ]:
ex.text_a

In [ ]:
ex.label, len(ex.label) 

In [ ]:
input_ids, input_mask, segment_ids, label_ids = samples_transformer(ex)

In [ ]:
input_ids

In [ ]:
input_mask

In [ ]:
segment_ids

In [ ]:
label_ids

In [ ]:
samples_transformer.label2id